<a href="https://colab.research.google.com/github/DanieleCecca/Interview-ArgoVision/blob/master/Classification_interview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificazione Animali

## Import and enironment setup

In [1]:
!pip install opencv-python -q

In [2]:
import cv2